6-7. 프로젝트: 멋진 작사가 만들기

<<목차>>

1) 데이터 읽어오기

2) 데이터 정제

3) 평가 데이터셋 분리

4) 인공지능 만들기

5) 회고

(1) 데이터 읽어오기

In [1]:
import glob
import tensorflow as tf
import os, re
import numpy as np
from sklearn.model_selection import train_test_split


txt_file_path = '/home/june/Lyrics/*.txt'
txt_list = glob.glob(txt_file_path)

raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw) 

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])


2022-10-12 20:46:04.489312: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-12 20:46:04.755633: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-12 20:46:05.539611: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.0/lib64:/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.7/lib64
2022-10-12 20:46:05.539682: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvi

데이터 크기: 187088
Examples:
 ["Let's stay together I, I'm I'm so in love with you", 'Whatever you want to do', 'Is all right with me']


각종 모듈을 불러오고 경로설정을 해준다.

extend() : 리스트함수로 추가적인 내용을 연장 한다.

read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. 

splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환


(2) 데이터 정제

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue # 길이가 0인 문장은 건너뜁니다.
    if sentence[-1] == ":": continue # 문장의 끝이 : 인 문장은 건너뜁니다.

    if idx >15: break
        
    print(sentence) # 일단 문장 10개만 확인해 볼 겁니다.

Let's stay together I, I'm I'm so in love with you
Whatever you want to do
Is all right with me
Cause you make me feel so brand new
And I want to spend my life with you Let me say that since, baby, since we've been together
Loving you forever
Is what I need
Let me, be the one you come running to
I'll never be untrue Oh baby
Let's, let's stay together (gether)
Lovin' you whether, whether
Times are good or bad, happy or sad
Oh, oh, oh, oh, yeah
Whether times are good or bad, happy or sad Why, why some people break up
Then turn around and make up
I just can't see


In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()   
    # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
     # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r'[" "]+', " ", sentence) 
     # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 

    sentence = sentence.strip()
    # 이전 스텝에서 본 것처럼 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 줍니다
    sentence = '<start> ' + sentence + ' <end>'      
    
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))   

<start> this is sample sentence . <end>


In [4]:
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> let s stay together i , i m i m so in love with you <end>',
 '<start> whatever you want to do <end>',
 '<start> is all right with me <end>',
 '<start> cause you make me feel so brand new <end>',
 '<start> and i want to spend my life with you let me say that since , baby , since we ve been together <end>',
 '<start> loving you forever <end>',
 '<start> is what i need <end>',
 '<start> let me , be the one you come running to <end>',
 '<start> i ll never be untrue oh baby <end>',
 '<start> let s , let s stay together gether <end>']

특수문자가 사라지고 공백들이 잘 정렬된 것을 확인.

이제 문장들을 토큰화하여 그 갯수가 15개가 넘는 문장들은 제외하여 학습시켜야한다.

In [5]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공합니다.
    # maxlen의 디폴트값은 None입니다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰집니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer
tensor, tokenizer = tokenize(corpus)

[[  16  222  283 ...   31    7    3]
 [   2  598    7 ...    0    0    0]
 [   2   26   24 ...    0    0    0]
 ...
 [   2   19  155 ...    0    0    0]
 [   2    5   32 ...    3    0    0]
 [   2 1137   10 ...    0    0    0]] <keras.preprocessing.text.Tokenizer object at 0x7f94b0616130>


In [6]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <END>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <START>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])

[ 16 222 283   5   4   5  22   5  22  30  14  33  31   7]
[222 283   5   4   5  22   5  22  30  14  33  31   7   3]


토큰화를 시켜주었다. 문자들이 이 토큰화되어 컴퓨터가 읽을 수 있도록 벡터변환 된것이다. 

(3) 평가 데이터셋 분리

In [10]:
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=32)

print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (140599, 14)
Target Train:  (140599, 14)


(4) 인공지능 만들기

In [19]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out


embedding_size = 32
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [20]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)   

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, batch_size=256)

Epoch 1/10
550/550 [==============================] - 115s 185ms/step - loss: 3.6910
Epoch 2/10
550/550 [==============================] - 102s 185ms/step - loss: 3.1331
Epoch 3/10
550/550 [==============================] - 102s 185ms/step - loss: 2.8387
Epoch 4/10
550/550 [==============================] - 102s 185ms/step - loss: 2.5531
Epoch 5/10
550/550 [==============================] - 102s 186ms/step - loss: 2.2656
Epoch 6/10
550/550 [==============================] - 102s 186ms/step - loss: 1.9914
Epoch 7/10
550/550 [==============================] - 102s 186ms/step - loss: 1.7539
Epoch 8/10
550/550 [==============================] - 102s 186ms/step - loss: 1.5561
Epoch 9/10
550/550 [==============================] - 102s 186ms/step - loss: 1.3984
Epoch 10/10
550/550 [==============================] - 102s 186ms/step - loss: 1.2753


In [21]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <END>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

인풋을 기초로 가사를 만들어내는 모델이다.

In [22]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you so much <end> '

I love로 시작하는 가사를 써달랬더니 이런 결과물을 주었다. 서윗한 가사다.

(5) 회고

- 이번 프로젝트에서 **어려웠던 점,**
    가장 NLP스러운 노드를 진행해본것 같다.  이해되지 않는 부분이 많았다.
    사실 이해가 되지 않는다기 보다는 그냥 무슨말인지 감도 오지 않았다.
    그래도 계속 읽고 또 읽다보니 서서히 이해가 되기 시작하였다. 이해하는데 오래 걸려서 힘들었다.

- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**
    이 노드 역시 데이터 전처리 부분이 굉장히 중요한 부분이라고 느꼈던게,  특수문자와 지나치게 긴 문장 제거 등 역시 데이터를 다듬고 정렬하고 바꿔주는 하는 것들이 예전 익스플로레이션 노드부터 계속 해왔고 계속 보아왔던 모습들이 NLP에서도 반복되었기 때문이다.
    다만 아직 데이터를 정제하고 전처리 하는 과정들의 기준을 어떤 데이터의 어디에 잡고 어떤 부분을 집중해야하고 하는 디테일한 과정은 아직 많은 과정이필요할 것 같다.

- 루브릭 평가 지표를 맞추기 위해 **시도한 것들**.
    첫 시도에서는 로스가 1.3대로 준수하게 나왔으나 새로운 시도에 2.2가 넘기에 임베딩 사이즈, 히든 사이즈, 배치 사이즈를 변경하여 시도하였다.


- 만약에 루브릭 평가 관련 지표를 **달성 하지 못했을 때, 이유에 관한 추정**


- **자기 다짐**

    개인적으로 많이 중요하다 생각 되었던 부분은 바로 데이터를 어떻게 기계가 이해, 내지 인식 시킬 수있을까였던 것 같다.
    토큰화로써 인간의 언어를 기계에 인식시킬 수가 있다는 사실이 신기했다.
    최신 기술이기에 앞으로 더 많은 발전으로 더욱어 진보된 AI 기술이 개발 될텐데 이 NLP 부분은 사람들에게 꾀나 편리함을 제공할 기술로 보인다. 
    NLP 에 대한 관심을 꾸준히 가지고 지켜봐야 할 것 같다.
